# Europe Health

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import os, shutil

## Open Questions

* Data Source
* Data current date

### Importing the data

In [ ]:
#description of the respective features
pd.set_option('display.max_colwidth', None)
desc_df = pd.read_csv('Supply_Food_Data_Descriptions.csv', index_col = 'Categories')
desc_df

In [ ]:
# change the directory to the one of your "food" folder here
files = os.listdir('/Users/franz/Desktop/DV Project/Dashboard/diet_corona/food/')
if '.DS_Store' in files:
    files.remove('.DS_Store')
directory = '/Users/franz/Desktop/DV Project/Dashboard/diet_corona/food/' + files[0]

this_df = pd.read_csv(directory)[["Area", "Item", "Value"]].set_index("Area", drop=True)
name = this_df.iloc[0,0]
this_df = this_df.rename(columns={"Value":name}).drop(columns="Item")
food_supply = this_df

for i in range(1,len(files)):
    directory = '/Users/franz/Desktop/DV Project/Dashboard/diet_corona/food/' + files[i]
    this_df = pd.read_csv(directory)[["Area", "Item", "Value"]].set_index("Area", drop=True)
    name = this_df.iloc[0,0]
    this_df = this_df.rename(columns={"Value":name}).drop(columns="Item")
    food_supply = pd.concat([food_supply, this_df], axis=1)


In [ ]:
smo = pd.read_csv("share-of-adults-who-smoke.csv")
#retrieved from https://ourworldindata.org/smoking
smo = smo[smo["Year"] == 2016].rename(columns={"Code":"iso_a3", "Smoking prevalence, total (ages 15+)":"Smoking"})
smo = smo[["Entity", "Smoking"]].set_index("Entity", drop = True)
food_supply = pd.concat([food_supply, smo], axis = 1)

In [ ]:
smo["Smoking"].std()

In [ ]:
obes = pd.read_csv('Food_Supply_Quantity_kg_Data.csv')[["Country", "Obesity"]].set_index("Country")
food_supply = pd.concat([food_supply, obes], axis = 1)
food_supply = food_supply.reset_index().rename(columns={"index":"Country"})

In [ ]:
exp = pd.read_excel("h_exp.xlsx")
#https://ec.europa.eu/eurostat/databrowser/view/HLTH_SHA11_HF__custom_227597/bookmark/table?lang=en&bookmarkId=1530a1e6-767e-4661-9e15-0ed2f7fae0d5

exp.head()

### Data Preprocessing

In [ ]:
#keep only countries from the EU
eu_countries_list = ["Austria","Belgium","Bulgaria","Croatia","Cyprus","Czechia","Denmark","Estonia","Finland","France",
                    "Germany","Greece","Hungary","Ireland","Italy","Latvia","Lithuania","Luxembourg","Malta","Netherlands","Poland",
                    "Portugal","Romania","Slovakia","Slovenia","Spain","Sweden"]

food_supply = food_supply[food_supply["Country"].isin(eu_countries_list)]

In [ ]:
def cz(s):
    if s == "Czech Republic":
        return "Czechia"
    elif s == "Slovak Republic":
        return "Slovakia"
    else:
        return s

exp["Country Name"] = exp["Country Name"].apply(cz)
exp = exp[exp["Country Name"].isin(eu_countries_list)]

len(exp)

In [ ]:
#check for nan values
if food_supply.dropna().shape == food_supply.shape:
    print("There are no nan values!")
else:
    print(food_supply.isna().sum())

In [ ]:
#drop columns with nans
food_supply = food_supply.drop(columns=["Sugar Crops", "Miscellaneous"])

### Assessing the variables

In [ ]:
print("There are", food_supply.shape[1], "different columns in the food_supply DataFrame!")
food_supply.columns

In [ ]:
food_supply.info()

### Adding Covid-19 realted Data

In [ ]:
# cases, deaths, tests, vaccinations

In [ ]:
vacc = pd.read_csv("https://covid.ourworldindata.org/data/owid-covid-data.csv?v=2021-03-23")
vacc.columns

In [ ]:
#retrieved from https://covid.ourworldindata.org/data/owid-covid-data.csv?v=2021-03-23
vacc = vacc[vacc["date"] == "2021-03-20"] 
vacc = vacc[vacc["location"].isin(eu_countries_list)]
vacc = vacc[["location", "total_cases_per_million", "total_deaths_per_million", "life_expectancy", "human_development_index", "population_density", "median_age", "gdp_per_capita", "cardiovasc_death_rate", "diabetes_prevalence"]] # , "total_tests_per_thousand", "total_vaccinations_per_hundred

In [ ]:
food_supply = food_supply.set_index("Country", drop=True)
exp = exp.set_index("Country Name", drop=True)
vacc = vacc.set_index("location", drop = True)
food_supply = pd.concat([food_supply, vacc, exp], axis = 1)
food_supply.head()

In [ ]:
#check for nan values
if food_supply.dropna().shape == food_supply.shape:
    print("There are no nan values!")
else:
    print(food_supply.isna().sum())

### Checking correlations

In [ ]:
def corr(df, color):
    fig = plt.figure(figsize=(20,16))
    mask = np.zeros_like(df.corr())
    mask[np.triu_indices_from(mask)] = True
    sns.heatmap(df.corr().round(2), annot=True, cmap=color, linewidths=0.2, mask=mask, vmin=-1, vmax=1)
    plt.xticks(fontsize=11)
    plt.yticks(fontsize=11)
    plt.savefig("correlation.pdf");

In [ ]:
food_supply = food_supply.rename(columns = {"life_expectancy":"Life Expectancy", "human_development_index":"Human Development Index",
                             "population_density":"Population Density", "median_age":"Median Age",
                             "gdp_per_capita":"GDP per Capita", "cardiovasc_death_rate":"Cardiovascular Death Rate",
                             "diabetes_prevalence":"Diabetes Prevalence"})

In [ ]:
corr(food_supply, "plasma")

### Feature engineering

In [ ]:
food_supply["Country"] = food_supply.index
food_supply.to_csv("scatter_data.csv")

# Part 1: Eating habits of the 27 EU countries - Map(s)

This part contains the per country per capita per year food consumption of different kinds of foods in kilogram. This includes the following 19 food categories:

In [ ]:
foods = food_supply.iloc[:,:20].columns.tolist()
foods

# Part 2: Correlation of the food habits and general information about the country

## Watch out for outliers!

This part investigates if there are any correlations between a countries eating habits and the general data about the country. This general data includes the following variables:
* GDP per capita
* Human Development Index
* Population Density
* Median Age

In [ ]:
def scatter_plot(df, x, y, size):
    '''insert x and y as string while x entails color coding (abv avg...)'''
    
    # feature engineering
    col_name = str(y) + " above avg"
    food_supply[col_name] = (df[y] > df[y].mean()).astype(int)
    
    size = df[size] * 1/4
    
    # plotting
    fig = px.scatter(df, x=x, y =y, size=size, color_continuous_scale=px.colors.sequential.Plasma, color=col_name,hover_name=df.index, log_x=False, trendline = "ols",  trendline_color_override="#bd3786",marginal_x = "box",marginal_y = "box", template="simple_white")
    fig.show()

In [ ]:
general_food_cor = food_supply.corr()[["GDP per Capita", "Human Development Index", "Median Age", "Population Density"]].T[foods]
fig = plt.figure(figsize=(20,5))
sns.heatmap(general_food_cor.round(2), annot=True, cmap="plasma", linewidths=0.2, vmin=-1, vmax=1)

In [ ]:
general_cor = food_supply[["GDP per Capita", "Human Development Index", "Median Age", "Population Density"]].corr()
general_cor

In [ ]:
scatter_plot(food_supply , "GDP per Capita", "Stimulants", "Human Development Index")

# Part 3: Correlation of the food habits and health information about the country

This part investigates if there are any correlations between a countries eating habits and the health data about the country. This data includes the following variables:
* Obesity
* Life Expectancy
* Cardiovasc Death Rate

In [ ]:
health_food_cor = food_supply.corr()[["Obesity", "Diabetes Prevalence", "Cardiovascular Death Rate", "Life Expectancy", "Health Expenditure" ]].T[foods].T
fig = plt.figure(figsize=(10,10))
sns.heatmap(health_food_cor.round(2), annot=True, cmap="plasma", linewidths=0.2, vmin=-1, vmax=1)#.set_yticklabels(["Obesity", "Diabetes Prevalence", "Cardiovascular Death Rate", "Life Expectancy", "Health Expenditure (% of GDP)"], rotation=0)

In [ ]:
df_corr_round = food_supply.corr()[["Obesity", "Diabetes Prevalence", "Cardiovascular Death Rate", "Life Expectancy", "Health Expenditure" ]].T[foods].T.round(2)
fig = ff.create_annotated_heatmap(
            z=df_corr_round.to_numpy(),
            x=df_corr_round.columns.tolist(),
            y=df_corr_round.index.tolist(),
            zmax=1, zmin=-1,
            showscale=True,
            hoverongaps=True,
            ygap=3
            )

fig.update_layout(yaxis_tickangle=-45, yaxis=dict(showgrid=False), xaxis=dict(showgrid=False))
# add title
fig.update_layout(title_text='<i><b>Correlation of Food Consumption and Health</b></i>',width=600, height=800)

In [ ]:
food_supply.columns

In [ ]:
#health_cor = food_supply[["Obesity", "life_expectancy", "cardiovasc_death_rate", "gdp_per_capita", "human_development_index", "median_age", "population_density"]].corr()[["Obesity", "life_expectancy", "cardiovasc_death_rate"]].T
#health_cor


In [ ]:
import plotly.graph_objs as go
import statsmodels.api as sm

df = food_supply.copy()
col_name = str("Fruits - Excluding Wine") + "(above Average)"
df[col_name] = (df["Fruits - Excluding Wine"] > df["Fruits - Excluding Wine"].mean())#.astype(int)

fig = px.scatter(df, x="Life Expectancy", y ="Fruits - Excluding Wine", size=df["GDP per Capita"], color=col_name,hover_name=df.index, log_x=False,marginal_x = "box",marginal_y = "box", template="simple_white", color_discrete_sequence=["#0d0887", "#9c179e"])

# linear regression
regline = sm.OLS(df["Fruits - Excluding Wine"],sm.add_constant(df["Life Expectancy"])).fit().fittedvalues

# add linear regression line for whole sample
fig.add_traces(go.Scatter(x=df["Life Expectancy"], y=regline,
                          mode = 'lines',
                          marker_color='#fb9f3a',
                          name='OLS Trendline')
                          )

fig.show()

In [ ]:
df = food_supply.copy()
col_name = str("Fruits - Excluding Wine") + ": above Average"
df[col_name] = (df["Fruits - Excluding Wine"] > df["Fruits - Excluding Wine"].mean())#.astype(int)

    
size = df["Cardiovascular Death Rate"]
    
fig = px.scatter(df, x="Life Expectancy", y ="Fruits - Excluding Wine", size=size, color=col_name,hover_name=df.index, log_x=False, trendline = "ols",  trendline_color_override="#bd3786",marginal_x = "box",marginal_y = "box", template="simple_white", color_discrete_sequence=px.colors.qualitative.G10)
fig.show()

In [ ]:
scatter_plot(food_supply , "Life Expectancy", "Fruits - Excluding Wine", "Cardiovascular Death Rate")

# Part 4: Correlation of the food habits and Covid-19 data about the country

This part investigates if there are any correlations between a countries eating habits and the Covid-19 data about the country. This data includes the following variables:
* relative Cases
* relative Deaths

In [ ]:
health_food_cor = food_supply.corr()[["total_cases_per_million", "total_deaths_per_million"]].T[foods]
fig = plt.figure(figsize=(17,5))
sns.heatmap(health_food_cor.round(2), annot=True, cmap="plasma", linewidths=0.2, vmin=-1, vmax=1).set_yticklabels(["total_cases_per_million", "total_deaths_per_million"], rotation=0)

In [ ]:
#health_cor = food_supply[["total_cases_per_million", "total_deaths_per_million", "Obesity", "life_expectancy", "cardiovasc_death_rate", "gdp_per_capita", "human_development_index", "median_age", "population_density"]].corr()[["total_cases_per_million", "total_deaths_per_million"]].T
#health_cor

In [ ]:
scatter_plot(food_supply , "total_cases_per_million", "Alcoholic Beverages", "total_deaths_per_million")

## Evtl.: Part 4.0: Covid - general timelines per chosen country

# Introducing health variables

In [ ]:
health = food_supply[["Country","Obesity", "Diabetes Prevalence", "Cardiovascular Death Rate", "Life Expectancy", "Health Expenditure" ]]



In [ ]:
health = food_supply[["Country","Obesity", "Diabetes Prevalence", "Cardiovascular Death Rate", "Life Expectancy", "Health Expenditure" ]]



fig = go.Figure()
fig.add_trace(go.Bar(
    x=health["Country"],
    y=health["Obesity"],
    name="Obesity",
    marker_color='#0d0887'
))
fig.add_trace(go.Bar(
    x=health["Country"],
    y=health["Diabetes Prevalence"],
    name="Diabetes Prevalence",
    marker_color='#7201a8'
))

fig.add_trace(go.Bar(
    x=health["Country"],
    y=health["Cardiovascular Death Rate"],
    name="Cardiovascular Death Rate",
    marker_color='#bd3786'
))

fig.add_trace(go.Bar(
    x=health["Country"],
    y=health["Life Expectancy"],
    name="Life Expectancy",
    marker_color='#ed7953'
))

fig.add_trace(go.Bar(
    x=health["Country"],
    y=health["Health Expenditure"],
    name="Health Expenditure (% of GDP)",
    marker_color='#fdca26'
))
# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.update_layout(plot_bgcolor='white')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', gridcolor='grey')
fig.update_xaxes(showline=True, linewidth=2, linecolor='black')



In [ ]:
health = food_supply[["Country","Obesity", "Diabetes Prevalence", "Cardiovascular Death Rate", "Life Expectancy", "Health Expenditure" ]]



fig = go.Figure()
fig.add_trace(go.Bar(
    x=health["Country"],
    y=np.log(health["Obesity"]),
    name="Obesity",
    marker_color='#0d0887'
))
fig.add_trace(go.Bar(
    x=health["Country"],
    y=np.log(health["Diabetes Prevalence"]),
    name="Diabetes Prevalence",
    marker_color='#7201a8'
))

fig.add_trace(go.Bar(
    x=health["Country"],
    y=np.log(health["Cardiovascular Death Rate"]),
    name="Cardiovascular Death Rate",
    marker_color='#bd3786'
))

fig.add_trace(go.Bar(
    x=health["Country"],
    y=np.log(health["Life Expectancy"]),
    name="Life Expectancy",
    marker_color='#ed7953'
))

fig.add_trace(go.Bar(
    x=health["Country"],
    y=np.log(health["Health Expenditure"]),
    name="Health Expenditure (% of GDP)",
    marker_color='#fdca26'
))
# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.update_layout(plot_bgcolor='white')
fig.update_yaxes(title_text="log")
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', gridcolor='grey')
fig.update_xaxes(showline=True, linewidth=2, linecolor='black')


In [ ]:
print("Hey",  "\<b> Hey \</b>" )

In [ ]:
print('\033[1m{:10s}\033[0m'.format('Hey'))

In [ ]:
BOLD = '\033[1m'
END = '\033[0m'
print('{}{}{}'.format(BOLD, "Hey", END))

In [ ]:
food = ["Alcoholic Beverages", "Animal fats", "Cereals - Excluding Beer", "Eggs", 
                "Fish, Seafood", "Fruits - Excluding Wine", "Meat", "Milk - Excluding Butter",
                "Offals", "Oilcrops", "Pulses", "Spices", "Starchy Roots","Stimulants", 
                "Sugar & Sweeteners", "Treenuts", "Vegetable Oils",
                "Vegetables" ]

In [ ]:
food.reverse()

In [ ]:
print(food)